## 模型融合

### 学习目标

- 学习融合策略
- 完成相应学习打卡任务

## 融合策略

### 简单加权融合

#### 平均法-Averaging
- 对于回归问题，一个简单直接的思路是取平均。将多个模型的回归结果取平均值作为最终预测结果，进而把多个弱分类器荣和城强分类器。
- 稍稍改进的方法是进行加权平均，权值可以用排序的方法确定，举个例子，比如A、B、C三种基本模型，模型效果进行排名，假设排名分别是1，2，3，那么给这三个模型赋予的权值分别是3/6、2/6、1/6。
- 平均法或加权平均法看似简单，其实后面的高级算法也可以说是基于此而产生的，Bagging或者Boosting都是一种把许多弱分类器这样融合成强分类器的思想。
- Averaging也可以用于对分类问题的概率进行平均

#### 投票法-voting

- 对于一个二分类问题，有3个基础模型，现在我们可以在这些基学习器的基础上得到一个投票的分类器，把票数最多的类作为我们要预测的类别。
- 投票法有硬投票（hard voting）和软投票（soft voting）
    - 硬投票: 对多个模型直接进行投票，不区分模型结果的相对重要度，最终投票数最多的类为最终被预测的类。
    - 软投票：增加了设置权重的功能，可以为不同模型设置不同权重，进而区别模型不同的重要度。

### stacking/blending

#### 堆叠法-stacking

- 用初始训练数据学习出若干个基学习器后，将这几个学习器的预测结果作为新的训练集(第一层)，来学习一个新的学习器(第二层)。

#### 混合法 - blending

- Blending与Stacking大致相同，只是Blending的主要区别在于训练集不是通过K-Fold的CV策略来获得预测值从而生成第二阶段模型的特征，而是建立一个Holdout集。简单来说，Blending直接用不相交的数据集用于不同层的训练。

#### 优缺点对比
- Blending的优点在于：

    - 比stacking简单（因为不用进行k次的交叉验证来获得stacker feature）

    - 避开了一个信息泄露问题：generlizers和stacker使用了不一样的数据集

    - 在团队建模过程中，不需要给队友分享自己的随机种子

- 而缺点在于：

    - 使用了很少的数据（是划分hold-out作为测试集，并非cv）

    - blender可能会过拟合（其实大概率是第一点导致的）

    - stacking使用多次的CV会比较稳健

## 相关代码

### 读取数据

In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import StratifiedKFold, KFold,train_test_split
from sklearn.ensemble import RandomForestClassifier 
import lightgbm as lgb
import catboost as cat
import xgboost as xgb
from catboost import Pool, CatBoostClassifier

In [7]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2 
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2 
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [17]:
data_path = '../team-learning-data-mining-master/wisdomOcean/data2/'
all_df = pd.read_csv(f'{data_path}group_df.csv',index_col=0)
all_df = reduce_mem_usage(all_df)
# all_df = all_df.fillna(99)

Memory usage of dataframe is 30.28 MB
Memory usage after optimization is: 7.59 MB
Decreased by 74.9%


In [25]:
all_df = all_df.fillna(99)

In [27]:
train = all_df[all_df['label'] != -1]
test =  all_df[all_df['label'] == -1]
feats = [c for c in train.columns if c not in ['ID', 'label']]

# 根据7：3划分训练集和测试集
X_train,X_val,y_train,y_val= train_test_split(train[feats],train['label'],test_size=0.3,random_state=0)

### 单模及加权融合

In [102]:
import pickle
def save_pickle(dic, save_path):
    print(f'Save {save_path} down...')
    with open(save_path, 'wb') as f:
        pickle.dump(dic, f)


def load_pickle(load_path):
    with open(load_path, 'rb') as f:
        message_dict = pickle.load(f)
    print(f'Finish read {load_path} ...')
    return message_dict


Save ./model/lgb_model.pkl down...


In [104]:
# 单模函数
def build_model_xgb(X_train,y_train):
    dtrain = xgb.DMatrix(X_train,label=y_train)
    dtest = xgb.DMatrix(X_train)

    num_round =1000
    param3 = {
    #         'verbosity':2 , 
    #         "tree_method":"gpu_hist"  ,  # 使用GPU
    #         "gpu_id": 0  , # 指定GPU
            'objective': 'multi:softmax' ,  # 目标 
            "num_class":3 ,  # 指定多分类别数目
            'eval_metric':'mlogloss', 
            "eta":0.01, 
            "gamma":0, 
            "max_depth":6, 
            "subsample":1 ,  # 横向抽样
            "colsample_bytree":1  , # 纵向抽样
            "colsample_bylevel":1, 
            "colsample_bynode":1, 
            "lambda":1 ,# L2, 
            "alpha":0 , # L1, 
    }

#     print("The time of param3:")
    model = xgb.train(params=param3, dtrain=dtrain, num_boost_round=num_round)
    pred = model.predict(dtest)
    print(f1_score(y_train,pred,average='macro'))
    save_pickle(model, './model/xgb_model.pkl')
    return model


def build_model_cat(X_train,y_train):
    train_dataset = Pool(data=X_train,
                     label=y_train,
#                      cat_features=cat_features
                    )
    eval_dataset = Pool(data=X_train,
                    label=y_train,
#                     cat_features=cat_features
                       )
    model = cat.CatBoostClassifier(
        iterations=1000,
        od_type='Iter',
        od_wait=120,
        max_depth=8,
        learning_rate=0.02,
        l2_leaf_reg=9,
        random_seed=2019,
        metric_period=50,
        fold_len_multiplier=1.1,
        loss_function='MultiClass',
        logging_level='Verbose'
        )
    model.fit(train_dataset)
    pred = model.predict(eval_dataset)
#     pred =[i[0]for i in pred]
    print(f1_score(y_train,pred,average='macro'))
    save_pickle(model, './model/cat_model.pkl')
    return model


def build_model_lgb(X_train,y_train):
    params = {  
        'boosting_type': 'gbdt',  
        'objective': 'multiclass',  
        'num_class': 3,  
        'metric': 'multi_error',  
        'num_leaves': 120,  
        'min_data_in_leaf': 100,  
        'learning_rate': 0.06,  
        'feature_fraction': 0.8,  
        'bagging_fraction': 0.8,  
        'bagging_freq': 5,  
        'lambda_l1': 0.4,  
        'lambda_l2': 0.5,  
        'min_gain_to_split': 0.2,  
        'verbose': -1, 
    }  
    print('Training...')
    trn_data = lgb.Dataset(X_train, y_train)
    test_data = lgb.Dataset(X_train,y_train )
    model = lgb.train(params, 
                    trn_data, 
                    num_boost_round = 1000,
    #                 valid_sets = [trn_data,val_data], 
    #                 verbose_eval = 100, 
    #                 early_stopping_rounds = 100
                     )
    pred = model.predict(X_train)
    pred = np.argmax(pred,axis = 1)
    print(f1_score(y_train,pred,average='macro'))
    save_pickle(model, './model/lgb_model.pkl')
    return model


In [105]:
ls model

lgb_model.pkl


In [106]:
model_lgb = build_model_lgb(X_train,y_train)
model_xgb = build_model_xgb(X_train,y_train)
model_cat = build_model_cat(X_train,y_train)

Training...
1.0
Save ./model/lgb_model.pkl down...
[18:35:03] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 90 extra nodes, 0 pruned nodes, max_depth=6
[18:35:03] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 66 extra nodes, 0 pruned nodes, max_depth=6
[18:35:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 102 extra nodes, 0 pruned nodes, max_depth=6
[18:35:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 88 extra nodes, 0 pruned nodes, max_depth=6
[18:35:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 70 extra nodes, 0 pruned nodes, max_depth=6
[18:35:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 104 extra nodes, 0 pruned nodes, max_depth=6
[18:35:05] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pru

[18:35:14] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 70 extra nodes, 0 pruned nodes, max_depth=6
[18:35:14] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 92 extra nodes, 0 pruned nodes, max_depth=6
[18:35:15] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 90 extra nodes, 0 pruned nodes, max_depth=6
[18:35:15] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 78 extra nodes, 0 pruned nodes, max_depth=6
[18:35:15] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 102 extra nodes, 0 pruned nodes, max_depth=6
[18:35:15] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 92 extra nodes, 0 pruned nodes, max_depth=6
[18:35:15] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 78 extra nodes, 0 pruned nodes, max_depth=

[18:35:23] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 106 extra nodes, 0 pruned nodes, max_depth=6
[18:35:23] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 86 extra nodes, 0 pruned nodes, max_depth=6
[18:35:23] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 80 extra nodes, 0 pruned nodes, max_depth=6
[18:35:23] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 104 extra nodes, 0 pruned nodes, max_depth=6
[18:35:23] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 88 extra nodes, 0 pruned nodes, max_depth=6
[18:35:24] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 78 extra nodes, 0 pruned nodes, max_depth=6
[18:35:24] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 108 extra nodes, 0 pruned nodes, max_dept

[18:35:31] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 82 extra nodes, 0 pruned nodes, max_depth=6
[18:35:31] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 88 extra nodes, 0 pruned nodes, max_depth=6
[18:35:31] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 112 extra nodes, 0 pruned nodes, max_depth=6
[18:35:32] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 96 extra nodes, 0 pruned nodes, max_depth=6
[18:35:32] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 88 extra nodes, 0 pruned nodes, max_depth=6
[18:35:32] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 114 extra nodes, 0 pruned nodes, max_depth=6
[18:35:32] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 90 extra nodes, 0 pruned nodes, max_depth

[18:35:40] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 86 extra nodes, 0 pruned nodes, max_depth=6
[18:35:40] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 112 extra nodes, 0 pruned nodes, max_depth=6
[18:35:41] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 96 extra nodes, 0 pruned nodes, max_depth=6
[18:35:41] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 82 extra nodes, 0 pruned nodes, max_depth=6
[18:35:41] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 108 extra nodes, 0 pruned nodes, max_depth=6
[18:35:41] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 98 extra nodes, 0 pruned nodes, max_depth=6
[18:35:41] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 84 extra nodes, 0 pruned nodes, max_depth

...

[18:41:01] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 44 extra nodes, 0 pruned nodes, max_depth=6
[18:41:01] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 26 extra nodes, 0 pruned nodes, max_depth=6
[18:41:01] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 82 extra nodes, 0 pruned nodes, max_depth=6
[18:41:01] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 44 extra nodes, 0 pruned nodes, max_depth=6
[18:41:02] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=6
[18:41:02] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 52 extra nodes, 0 pruned nodes, max_depth=6
[18:41:02] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 44 extra nodes, 0 pruned nodes, max_depth=6

[18:41:08] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 38 extra nodes, 0 pruned nodes, max_depth=6
[18:41:08] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 38 extra nodes, 0 pruned nodes, max_depth=6
[18:41:08] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 58 extra nodes, 0 pruned nodes, max_depth=6
[18:41:08] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=6
[18:41:09] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=6
[18:41:09] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=6
[18:41:09] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 50 extra nodes, 0 pruned nodes, max_depth=6

[18:41:15] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_depth=6
[18:41:15] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 80 extra nodes, 0 pruned nodes, max_depth=6
[18:41:15] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=6
[18:41:16] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 68 extra nodes, 0 pruned nodes, max_depth=6
[18:41:16] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 34 extra nodes, 0 pruned nodes, max_depth=6
[18:41:16] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=6
[18:41:16] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 26 extra nodes, 0 pruned nodes, max_depth=6

[18:41:22] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=6
[18:41:22] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 56 extra nodes, 0 pruned nodes, max_depth=6
[18:41:22] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 32 extra nodes, 0 pruned nodes, max_depth=6
[18:41:23] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 40 extra nodes, 0 pruned nodes, max_depth=6
[18:41:23] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 72 extra nodes, 0 pruned nodes, max_depth=6
[18:41:23] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=6
[18:41:23] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=6

[18:41:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 34 extra nodes, 0 pruned nodes, max_depth=6
[18:41:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=6
[18:41:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 44 extra nodes, 0 pruned nodes, max_depth=6
[18:41:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 40 extra nodes, 0 pruned nodes, max_depth=6
[18:41:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=6
[18:41:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 40 extra nodes, 0 pruned nodes, max_depth=6
[18:41:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 84 extra nodes, 0 pruned nodes, max_depth=6

In [114]:
model_xgb = load_pickle('./model/xgb_model.pkl')
model_lgb = load_pickle('./model/lgb_model.pkl')
model_cat = load_pickle('./model/cat_model.pkl')

Finish read ./model/xgb_model.pkl ...
Finish read ./model/lgb_model.pkl ...
Finish read ./model/cat_model.pkl ...


In [141]:
pred_xgb = model_xgb.predict(xgb.DMatrix(X_val))


pred_lgb = model_lgb.predict(X_val)
pred_lgb = np.argmax(pred_lgb,axis = 1)

eval_dataset = Pool(data=X_val,)
pred_cat = model_cat.predict(eval_dataset)
pred_cat = [i[0] for i in pred_cat]

In [142]:
X_val['pred_xgb'] = pred_xgb
X_val['pred_lgb'] = pred_lgb
X_val['pred_cat'] = pred_cat


In [152]:
X_val['val_voting'] = X_val.loc[:,['pred_xgb','pred_lgb','pred_cat']].mode(axis = 1)[0]

In [155]:
voting_f1_score = f1_score(y_val,X_val['val_voting'],average='macro')
print(voting_f1_score)

0.897016720213342


In [157]:
_N_FOLDS = 5  # 采用5折交叉验证
from sklearn.model_selection import StratifiedKFold, KFold,train_test_split
kf = KFold(n_splits=_N_FOLDS, shuffle = True,random_state=42)  # sklearn的交叉验证模块，用于划分数据


def get_oof(clf, X_train, y_train, X_test):
    oof_train = np.zeros((X_train.shape[0], 1)) 
    oof_test_skf = np.empty((_N_FOLDS, X_test.shape[0], 1))  
    
    for i, (train_index, test_index) in enumerate(kf.split(X_train)): # 交叉验证划分此时的训练集和验证集
        kf_X_train = X_train.iloc[train_index,]
        kf_y_train = y_train.iloc[train_index,]
        kf_X_val = X_train.iloc[test_index,]
        
        clf.fit(kf_X_train, kf_y_train)
        oof_train[test_index] = clf.predict(kf_X_val).reshape(-1, 1) 
        oof_test_skf[i, :] = clf.predict(X_test).reshape(-1, 1)  
    oof_test = oof_test_skf.mean(axis=0)  # 对每一则交叉验证的结果取平均
    return oof_train, oof_test  # 返回当前分类器对训练集和测试集的预测结果

In [ ]:
X_train

In [159]:
# 将你的每个分类器都调用get_oof函数，并把它们的结果合并，就得到了新的训练和测试数据new_train,new_test
new_train, new_test = [], []


model1 = RandomForestClassifier(n_estimators = 100)
model2 = lgb.LGBMClassifier(num_leaves=127,learning_rate = 0.1,n_estimators = 200)
model3 = lgb.LGBMClassifier(num_leaves=63,learning_rate = 0.05,n_estimators = 400)

for clf in [model1, model2, model3]:
    oof_train, oof_test = get_oof(clf, X_train[feats], y_train, X_val[feats])
    new_train.append(oof_train)
    new_test.append(oof_test)
    
new_train = np.concatenate(new_train, axis=1)
new_test = np.concatenate(new_test, axis=1)

In [173]:
# 用新的训练数据new_train作为新的模型的输入，stacking第二层
# 使用LogisticRegression作为第二层是为了防止模型过拟合
# 这里使用的模型还有待优化，因此模型融合效果并不是很好
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(new_train, y_train)
result = clf.predict(new_test)

stacking_f1_score = f1_score(y_val,result,average='macro')
print(stacking_f1_score)

0.8753250818861686


### 思考题

- 如何基于stacking改进出blending - stacking使用了foldCV，blending使用了holdout.
    - 

- stacking还可以进行哪些优化提升F1-score - 从第一层模型数量？模型差异性？角度出发
    -  在保证模型计算效率的情况下，模型的数量尽可能的多  
    - 模型的差异性竟可能的大
        - 模型的结构尽可能丰富多样，使用树模型，线性模型，神经网络模型
        - 各个模型的特征不一样，多套特征工程
        - 同种算法随机种子也可以设置不一样

### 参考资料

[XGBoost：参数解释](https://blog.csdn.net/zc02051126/article/details/46711047)

[Catboost参数详解](ils/111252715?utm_medium=distribute.pc_relevant_download.none-task-blog-baidujs-1.nonecase&depth_1-utm_source=distribute.pc_relevant_download.none-task-blog-baidujs-1.nonecase)
